In [1]:
from robotic import ry

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.view()

0

In [3]:
obj = C.addFrame('obj','table') \
    .setShape(ry.ST.capsule, size=[1.,.03]) \
    .setColor([1,.5,0])
obj.setRelativePose("t(-.4 -.4 0.5) ") #d(30 0 1 0) d(45 0 0 1)

# set to BOX

#obj.setShape(ry.ST.ssBox, size=[.05, .1, 1., .005])
#C.view()
#C.addFrame('mark','obj') \
#.setShape(ry.ST.marker, size=[.2,.2])#red=x, green=y, blue=z

###
C.view()

0

In [66]:
komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(1., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);

In [199]:
# SIMPLE CYLINDER GRASP
komo.addObjective([], ry.FS.positionDiff, ['l_gripper', 'obj'], ry.OT.eq, [1e1]);
komo.addObjective([], ry.FS.vectorY, ['l_gripper'], ry.OT.eq, [0,0,1]); #vectorZ works here

In [109]:
# RELAXED ORIENTATION CYLINDER GRASP
komo.addObjective([], ry.FS.positionDiff, ['l_gripper', 'obj'], ry.OT.eq, [1e1]);
komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper','obj'], ry.OT.eq, [1e1]);
komo.addObjective([], ry.FS.distance, ['l_palm','obj'], ry.OT.ineq, [1e1]);

In [203]:
# RELAXED POSITION & ORIENTATION CYLINDER GRASP
komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'obj'], ry.OT.eq, [[10,0,0],[0,10,0]]);
komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'obj'], ry.OT.ineq, [0,0,10], target=[0,0,0.5]);
komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'obj'], ry.OT.ineq, [0,0,-10], target=[0,0,-0.5]);
komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper','obj'], ry.OT.eq, [1e1]);
komo.addObjective([], ry.FS.distance, ['l_palm','obj'], ry.OT.ineq, [1e1]);

In [327]:
# BOX GRASP
length_x = obj.getSize()[0] # 0.05
length_y = obj.getSize()[1] # 0.1
length_z = obj.getSize()[2] # 1…0

length_gripper = C.getFrame('l_gripper').getSize() # 0.03
komo.addObjective([], ry.FS.distance, ['l_palm','obj'], ry.OT.ineq, [1e1], target=[-0.00]);

def ineqSliding(axis):
    if (axis == "x"): target = [0,0,length_x/2]; factor = [10,0,0]
    if (axis == "y"): target = [0,0,length_y/2]; factor = [0,10,0]
    if (axis == "z"): target = [0,0,length_z/2]; factor = [0,0,10]
    komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'obj'], ry.OT.ineq, factor, target=target);
    komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'obj'], ry.OT.ineq, factor*-1, target=target*-1);

if(length_x < length_gripper*2):
    komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'obj'], ry.OT.eq, [[10,0,0]]);
    
    ineqSliding("y")
    ineqSliding("z")
    
    komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper','obj'], ry.OT.eq, [1e1]);
    komo.addObjective([], ry.FS.scalarProductXY, ['l_gripper','obj'], ry.OT.eq, [1e1]);
    print("GRASP ON X AXIS")
    
elif(length_y < length_gripper*2):
    komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'obj'], ry.OT.eq, [[0,10,0]]);
    
    ineqSliding("x")
    ineqSliding("z")
    
    komo.addObjective([], ry.FS.scalarProductXX, ['l_gripper','obj'], ry.OT.eq, [1e1]);
    komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper','obj'], ry.OT.eq, [1e1]);
    print("GRASP ON Y AXIS")
elif(length_z < length_gripper*2):
    
    komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'obj'], ry.OT.eq, [[0,0,10]]);
    
    ineqSliding("x")
    ineqSliding("y")
    
    komo.addObjective([], ry.FS.scalarProductXY, ['l_gripper','obj'], ry.OT.eq, [1e1]);
    komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper','obj'], ry.OT.eq, [1e1]);
    print("GRASP ON Z AXIS")


GRASP ON X AXIS


In [12]:
# PUSH
length_x = obj.getSize()[0] # 0.05
length_y = obj.getSize()[1] # 0.1
length_z = obj.getSize()[2] # 1…0
# define push vector
#push = C.addFrame('mark','obj') \
#    .setShape(ry.ST.marker, size=[.1,.1])
#push.setRelativePose("t(.5 .5 .5 ) d(90 0 1 0)")
def ineqSliding(axis):
    if (axis == "x"): target = [0,0,length_x/2]; factor = [10,0,0]
    if (axis == "y"): target = [0,0,length_y/2]; factor = [0,10,0]
    if (axis == "z"): target = [0,0,length_z/2]; factor = [0,0,10]
    komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'obj'], ry.OT.ineq, factor, target=target);
    komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'obj'], ry.OT.ineq, factor*-1, target=target*-1);

push = [0,1,0]

# define constraints
komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(1., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);

komo.addObjective([], ry.FS.positionRel, ['l_gripper', 'obj'], ry.OT.eq, [1], target=);

komo.addObjective([], ry.FS.vectorY, ['obj'], ry.OT.eq, [1e1], target = push);

# 1. scalarProductXY = 0 AND scalarProductXZ = 0 OR scalarProductXX = 0 AND scalarProductXZ = 0
# 2. 

In [13]:
ret = ry.NLP_Solver() \
    .setProblem(komo.nlp()) \
    .setOptions( stopTolerance=1e-2, verbose=4 ) \
    .solve()
print(ret)

{ time: 0.009313, evals: 23, done: 1, feasible: 0, sos: 1.91765, f: 0, ineq: 3.26352e-05, eq: 20.0002 }====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):270.23 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    252.942  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    237.523  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    225.444  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):     215.71  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    207.484  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):    203.415  ACCEPT
--newton-- it:   7  |Delta|:        0.2  alpha:          1  evals:   8  f(y):    202.432  ACCEPT
--newton-- it:   8  |Delta|:        0.2  alpha:          1  evals:   9

In [14]:
komo.view(False, "IK solution")

0

In [85]:
q = komo.getPath()
print(type(q), len(q))

In [86]:
C.setJointState(q[0])
C.view()

In [ ]:
del komo
del C